In [4]:
CREATE TABLE Products(
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
);

SyntaxError: invalid syntax (791921442.py, line 1)

In [1]:
import sqlite3
import time

##Nasser Alhalidi,Abdulaziz badr,Fahad Abdulallah,Abdulallah.

def execute_with_retry(cursor, query, params=(), retries=5):
    for i in range(retries):
        try:
            cursor.execute(query, params)
            return
        except sqlite3.OperationalError as e:
            if "locked" in str(e):
                wait_time = (i + 1) * 2  # increasing wait time
                time.sleep(wait_time)  # wait before retrying
            else:
                raise
    raise Exception("Database is locked after multiple retries")

# Connect to the database
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create the Books table
execute_with_retry(cursor, '''
CREATE TABLE IF NOT EXISTS Books (
    id INT PRIMARY KEY,
    title VARCHAR(255),
    author VARCHAR(255),
    publication_year INT
);
''')

# Create the Members table
execute_with_retry(cursor, '''
CREATE TABLE IF NOT EXISTS Members (
    member_id INT PRIMARY KEY,
    name VARCHAR(255),
    date_of_membership DATE
);
''')

# Create the BorrowingHistory table
execute_with_retry(cursor, '''
CREATE TABLE IF NOT EXISTS BorrowingHistory (
    borrow_id INT PRIMARY KEY,
    book_id INT,
    member_id INT,
    borrow_date TEXT,
    return_date TEXT,
    FOREIGN KEY (book_id) REFERENCES Books(id),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
);
''')

# Add a new book with a different id
execute_with_retry(cursor, '''
    INSERT INTO Books (id, title, author, publication_year) VALUES (?, ?, ?, ?)
''', (2, 'the art of war', 'Sun Tzu', 2009))

# Add a new member with a different member_id
execute_with_retry(cursor, '''
    INSERT INTO Members (member_id, name, date_of_membership) VALUES (?, ?, ?)
''', (2, 'Abdulaziz', '2010-02-01'))

# Add a new borrowing with different borrow_id, book_id, and member_id
execute_with_retry(cursor, '''
    INSERT INTO BorrowingHistory (borrow_id, book_id, member_id, borrow_date, return_date) VALUES (?, ?, ?, ?, ?)
''', (2, 2, 2, '2024-01-01', '2024-01-10'))

# Display the Books table
execute_with_retry(cursor, 'SELECT * FROM Books')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Display the Members table
execute_with_retry(cursor, 'SELECT * FROM Members')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Display the BorrowingHistory table
execute_with_retry(cursor, 'SELECT * FROM BorrowingHistory')
rows = cursor.fetchall()
for row in rows:
    print(row)

# SQL Injection example (unsafe)
unsafe_input = "' OR '1'='1"
query = f"SELECT * FROM Members WHERE name = '{unsafe_input}'"
execute_with_retry(cursor, query)
rows = cursor.fetchall()
print("Results of SQL Injection attempt:")
for row in rows:
    print(row)

# Fix SQL Injection using parameterized queries
safe_input = "' OR '1'='1"
execute_with_retry(cursor, "SELECT * FROM Members WHERE name = ?", (safe_input,))
rows = cursor.fetchall()
print("Results after fixing SQL Injection:")
for row in rows:
    print(row)

# Commit the changes and close the connection
conn.commit()
conn.close()


(2, 'the art of war', 'Sun Tzu', 2009)
(2, 'Abdulaziz', '2010-02-01')
(2, 2, 2, '2024-01-01', '2024-01-10')
Results of SQL Injection attempt:
(2, 'Abdulaziz', '2010-02-01')
Results after fixing SQL Injection:
